In [21]:
import gymnasium as gym
import os
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
from tqdm import tqdm
from collections import deque, defaultdict
import seaborn as sns
import numpy as np


In [25]:
class Actor(nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = nn.Linear(self.state_size, 128)
        self.linear2 = nn.Linear(128, 256)
        self.linear3 = nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = Categorical(F.softmax(output, dim=-1))
        return distribution

class Critic(nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = nn.Linear(self.state_size, 128)
        self.linear2 = nn.Linear(128, 256)
        self.linear3 = nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value


def compute_returns(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns


def trainIters(env, actor, critic, n_iters, device):
    optimizerA = optim.Adam(actor.parameters())
    optimizerC = optim.Adam(critic.parameters())
    loop = tqdm(range(n_iters))
    reward_window = deque(maxlen=100)
    history = defaultdict(list)

    for iter in loop:
        state, info = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        done = False
        total_reward = 0

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, terminated, truncated, info = env.step(action.cpu().numpy())
            total_reward += reward

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            if terminated or truncated:
                done = True

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                # print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = compute_returns(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()

        reward_window.append(total_reward)

        avg_reward = np.mean(reward_window)

        history['reward'].append(avg_reward)

        loop.set_description(f'avg reward = {avg_reward}')

    env.close()
    return history


In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("LunarLander-v2").unwrapped

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001
epochs = 10000


actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
history = trainIters(env, actor, critic, n_iters=epochs, device = device)

  0%|          | 0/10000 [00:00<?, ?it/s]/Users/jamesnguyen/anaconda3/envs/torch/lib/python3.9/site-packages/gymnasium/envs/box2d/lunar_lander.py:606: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("LunarLander-v2", render_mode="rgb_array")
  gym.logger.warn(
avg reward = -55.46798511919255:   6%|▌         | 608/10000 [05:35<1:26:16,  1.81it/s] 


KeyboardInterrupt: 